In [1]:
# importamos simpy
import simpy
# importamos random
import random

In [2]:
import pandas as pd

# Primera Simulación con 25 procesos

In [3]:
# creamos una lista para guardar los datos
lista = []

In [4]:
# definimos los programas
def Programa(nombre, env, creacionproceso, CPU):
    # damos un tiempo random antes de llegar al cpu
    yield env.timeout(creacionproceso)
    # llegada al cpu
    horallegada = env.now
    # definimos el runtime con un random entre 1 y 10
    runtime = random.randint(1,11)
    # mostramos un mensaje al usuario con los detalles
    print('%s llega a las %f y requiere %d instrucciones'%(nombre, horallegada,runtime))
    # solicitamos el cpu
    with CPU.request() as turno:
        #produce la solicitud
        yield turno
        # completa las instrcciones en el cpu 
        yield env.timeout(runtime)
        # guardamos la hora de salida
        horasalida = env.now
        # desplegamos un mensaje al usuario
        print('%s sale del cpu a las %f'%(nombre, horasalida))
    
    # calculamos el tiempo total para completar el proceso
    tiempototal = horasalida-horallegada
    # agregamos el tiempo total a la lista 
    lista.append(tiempototal)
        
    
    

In [5]:
# creamos el ambiente 
env = simpy.Environment()
# creamos el recurso CPU 
CPU = simpy.Resource(env, capacity =1)
# random seed
random.seed(10)

In [6]:
# fijamos la cantidad de programas a simular 
for i in range(25):
    env.process(Programa('Programa numero %d'%i,env, random.expovariate(1/10),CPU))
    
env.run()

Programa numero 13 llega a las 0.455795 y requiere 8 instrucciones
Programa numero 22 llega a las 1.426911 y requiere 10 instrucciones
Programa numero 7 llega a las 1.746267 y requiere 7 instrucciones
Programa numero 3 llega a las 2.307955 y requiere 1 instrucciones
Programa numero 10 llega a las 2.876776 y requiere 10 instrucciones
Programa numero 17 llega a las 3.335420 y requiere 1 instrucciones
Programa numero 9 llega a las 3.971589 y requiere 4 instrucciones
Programa numero 16 llega a las 4.806295 y requiere 3 instrucciones
Programa numero 1 llega a las 5.601718 y requiere 4 instrucciones
Programa numero 19 llega a las 6.103351 y requiere 5 instrucciones
Programa numero 8 llega a las 7.353646 y requiere 9 instrucciones
Programa numero 13 sale del cpu a las 8.455795
Programa numero 0 llega a las 8.472372 y requiere 6 instrucciones
Programa numero 2 llega a las 8.629663 y requiere 4 instrucciones
Programa numero 15 llega a las 9.242992 y requiere 6 instrucciones
Programa numero 6 ll

In [7]:
datos = {'25 procesos':lista}

dataframe = pd.DataFrame(datos)

In [8]:
dataframe


,25 procesos
0,8.000000
1,17.028884
2,23.709528
3,24.147840
4,33.579019
5,34.120375
6,37.484206
7,39.649501
8,42.854077
9,47.352445


In [10]:
dataframe["25 procesos"].mean()

68.78959825209033

In [11]:
dataframe["25 procesos"].std()

37.19360415080694